In [20]:
import os
import csv
import torch
import librosa
from datasets import Dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
from jiwer import wer
import evaluate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import TrainingArguments, Trainer
from P2_Transformers_Speech_Classification.project_2 import predict_single_audio, create_dataset

In [2]:
os.getcwd()

'C:\\Users\\wojew\\Documents\\DS\\sem1\\Deep Learning\\CINIC10_Proj1\\P2_Transformers_Speech_Classification'

# Load model

In [45]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

## Predict single audio

In [3]:
predict_single_audio(model, processor, 'data/train/audio/happy/0ab3b47d_nohash_0.wav')

Transcription for data/train/audio/happy/0ab3b47d_nohash_0.wav: HAPPY


'HAPPY'

# Create dataset

In [10]:
audio_dir = "data/train/audio/" 
dataset = create_dataset(audio_dir, processor, 'my_list2')

s
s
s
s
s
s
s
s
s


Dataset({
    features: ['input_values', 'labels'],
    num_rows: 9
})

# Predict on dataset

In [53]:
predictions = []
ground_truths = []
for audio_file in dataset:
    logits = model(input_values=audio_file["input_values"]).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    ground_truths.append(audio_file["labels"])
    predictions.append(transcription)

# Calculate Word Error Rate (WER) and accuracy
ground_truths = dataset["labels"]  # Assuming transcriptions are available
wer_score = wer(ground_truths, predictions)
accuracy = sum(1 for gt, pred in zip(ground_truths, predictions) if gt == pred) / len(ground_truths)

print("WER:", wer_score)
print("Accuracy:", accuracy)

TypeError: list indices must be integers or slices, not tuple

# Fine-tune model

In [11]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments

model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
model.freeze_feature_extractor()


# def prepare_dataset(example):
#     input_values = processor(example["input_values"], return_tensors="pt").input_values
#     labels = processor(example["labels"], return_tensors="pt", padding='longest').input_values
#     return {"input_values": input_values, "labels": labels}
# 
# encoded_dataset = dataset.map(prepare_dataset)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [54]:
training_args = TrainingArguments(
    output_dir="./output_dir",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    save_strategy='epoch',
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    labels = [list(item[0]) for item in labels]
    # print(labels)
    # print(type(labels))
    # print(type(labels[0]))
    # print(type(labels[0][0]))
    if all(isinstance(i, list) for i in preds):
        preds = [item for sublist in preds for item in sublist]
    pred_transcriptions = processor.batch_decode(preds)
    true_transcriptions = processor.batch_decode(labels)
    print(pred_transcriptions, true_transcriptions)
    wer_score = wer(true_transcriptions, pred_transcriptions)

    correct_predictions = sum([true == pred for true, pred in zip(true_transcriptions, pred_transcriptions)])
    total_predictions = len(true_transcriptions)
    accuracy = correct_predictions / total_predictions

    return {"wer": wer_score, "accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=4, training_loss=19.222543716430664, metrics={'train_runtime': 24.7232, 'train_samples_per_second': 0.728, 'train_steps_per_second': 0.162, 'total_flos': 163116840960000.0, 'train_loss': 19.222543716430664, 'epoch': 2.0})

In [43]:
trainer.save_model("./model_test3")
model = Wav2Vec2ForCTC.from_pretrained("./model_test3")

In [44]:
predict_single_audio(model, processor, 'data/train/audio/happy/0ab3b47d_nohash_0.wav')

Transcription for data/train/audio/happy/0ab3b47d_nohash_0.wav: ABBY


'ABBY'

In [9]:
predict_single_audio(model, processor, 'data/train/audio/bird/ff63ab0b_nohash_0.wav')

Transcription for data/train/audio/bird/ff63ab0b_nohash_0.wav: BAR


'BAR'

In [17]:
predict_single_audio(model, processor, 'data/train/audio/bird/ff63ab0b_nohash_0.wav')

Transcription for data/train/audio/bird/ff63ab0b_nohash_0.wav: BIRD


'BIRD'

In [42]:
metrics = trainer.evaluate(dataset)

print(metrics)

['', '', '', 'BIRD', 'RD', 'CAT', 'CAT', 'DOG', 'BIRD'] ['BED', 'BED', 'BIRD', 'BIRD', 'BIRD', 'CAT', 'CAT', 'DOG', 'BIRD']
{'eval_loss': -32.08187484741211, 'eval_wer': 0.4444444444444444, 'eval_accuracy': 0.5555555555555556, 'eval_runtime': 1.9817, 'eval_samples_per_second': 4.542, 'eval_steps_per_second': 1.009, 'epoch': 2.0}


In [38]:
metrics = trainer.evaluate(dataset)

print(metrics)

{'eval_loss': -37.5993537902832, 'eval_runtime': 1.6182, 'eval_samples_per_second': 5.562, 'eval_steps_per_second': 1.236, 'epoch': 2.0}
